### Meteo Station Utils

In [2]:
def order_meteo_zip(path, folder):
    ### access files 
    ziplist = glob.glob(f'{path}/*.zip')
    
    ### access folder
    zf = zipfile.ZipFile(f'{ziplist[folder]}')
    
    #extract dates of each of the zipfiles from file name
    a = []
    for i in range(len(zf.namelist())):
        date_str = zf.namelist()[i].split('/')[1]
        try:
            date_date = datetime.strptime(date_str, '%Y-%m-%d')
        except:
            continue
        a.append(date_date)
        
    # get unique values
    unique_dates = list(set(a))
    # order by date
    unique_dates.sort()
    #convert back to string
    str_dates = []
    for l in range(len(unique_dates)):
        str_dates.append(unique_dates[l].strftime('%Y-%m-%d'))
        
    ## Reorder files
    files_by_date = []
    for dates in str_dates:
        #r = re.compile(f'.*/{dates}/.*') # all files (including excel sheets)
        r = re.compile(f'.*/{dates}/.*.csv') # only csv files
        newlist = list(filter(r.match, zf.namelist())) 
        files_by_date.extend(newlist)
    
    return files_by_date

In [3]:
def read_meteo_csv(path, folder, file): 
    ### access files 
    ziplist = glob.glob(f'{path}/*.zip')
    
    ### access folder
    zf = zipfile.ZipFile(f'{ziplist[folder]}')
    
    try:
        df = pd.read_csv(zf.open(file),header=None, encoding = 'latin-1')
        print('Success regular csv, no header, latin encoding')
    except:
        try:
            df = pd.read_csv(zf.open(file), sep = ';', encoding = 'latin-1')
            if len(df.columns) == 1:
                df = df[df.columns[0]].str.split(',',expand=True)
                df = df.iloc[1:]
            print('Success sep ";", latin encoding')
        except:
            print('Failed')
    
    ## Detect and elminate empty columns- incluiding columns with low number of values (<1% of file)
    df = df.replace('', np.nan)
    empty_cols = [col for col in df.columns if df[col].isnull().all() or round(df[col].isnull().value_counts()[0]/len(df[col]),2) < 0.01]
    df.drop(empty_cols,
        axis=1,
        inplace=True)
    if len(df.columns)==5:
        df.columns = ['Date', 'Time','Variable', 'Value','Type']
    
    # add frequency as a variable
    freq = ['1min','5min','1hora']
    df['Freq']= re.findall('|'.join(freq),files_by_date[4])[0]
    print(f'file ={file}, cols = {len(df.columns)}')
    return df

In [ ]:
### Get unique variable names
def get_unique_variables(files_by_date, export_name):
    summary_df = pd.DataFrame(columns = ['station','filename','colnum','colnames','nrows'])
    for i in range(0,len(files_by_date)):
    
        df = read_meteo_csv(path =path,folder = 0,file =files_by_date[i])
        if len(df.columns)==6:
            colname = [list(df.Variable.unique())]
        else:
            colname= [list(df.columns)]
        try:
            filename=zip.namelist()[i].split('/')[2]
        except:
            filename=zip.namelist()[i].split('/')[1]
        fill = {'station':zip.namelist()[i].split('/')[0].split(' ')[2],
            'filename':filename,
            'colnum':len(df.columns),
            'colnames':colname,
            'nrows':len(df)}
        summary_df = summary_df.append(fill, ignore_index=True)
    summary_df.to_csv(export_name) 
        
    all_var = []
    for i in range(0,len(summary_df)):
        file_vars= summary_df.colnames[i][0]
        not_in_list =list(set(file_vars) - set(all_var))
        all_var.extend(not_in_list) 
    all_var = sorted([str(i) for i in all_var])
    
    return all_var